# Matrices de Transición por Ciclo de Sueño

Este notebook analiza cómo cambian las **transiciones entre fases del sueño** a lo largo de la noche, calculando matrices de transición separadas para cada ciclo de sueño.

## Objetivos

1. **Particionar por ciclos**: Usar la función **detectar_ciclos_rem** para dividir cada hipnograma en ciclos.
2. **Matrices por ciclo**: Calcular matrices de transición para Ciclo 1, Ciclo 2, Ciclo 3, etc.
3. **Análisis comparativo**: Ver si la probabilidad de transición a REM o SWS cambia conforme avanza la noche.
4. **Visualización**: Generar gráficos que muestren la evolución de las transiciones clave (S2 → REM, S2 → S3/S4) a través de los ciclos.

In [1]:
from pathlib import Path
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# Configuración (consistente con otros notebooks)
RUTA_DATOS = Path("../Datos")

ETIQUETAS_FASES = {
    0: "Vigilia (W)",
    1: "S1",
    2: "S2",
    3: "S3/S4",
    4: "REM",
    5: "Movimiento/SC"
}

def agrupar_fase(valor):
    """Mapea cualquier fase cruda al alfabeto reducido."""
    return {4: 3, 5: 4, 6: 5, 7: 5}.get(valor, valor)


def remapear_secuencia(secuencia):
    """Aplica `agrupar_fase` a cada época para unificar símbolos antes del conteo."""
    return [agrupar_fase(valor) for valor in secuencia]

print("Configuración cargada.")

Configuración cargada.


In [3]:
def listar_pacientes(ruta_datos=None):
    """Devuelve los IDs disponibles (archivos `Scoring_*_2`)."""
    ruta_datos = ruta_datos or RUTA_DATOS
    pacientes = []
    for archivo in ruta_datos.iterdir():
        nombre = archivo.name
        if archivo.is_file() and nombre.startswith("Scoring_") and nombre.endswith("_2"):
            pacientes.append(nombre.replace("Scoring_", ""))
    pacientes.sort()
    if not pacientes:
        raise RuntimeError("No se encontraron archivos `Scoring_*_2` en la ruta indicada.")
    return pacientes


def cargar_hipnograma(paciente, ruta_datos=None):
    """Carga la columna `scor_clean` y la devuelve como lista de enteros."""
    ruta_datos = ruta_datos or RUTA_DATOS
    ruta_archivo = ruta_datos / f"Scoring_{paciente}"
    columnas = ["epoca", "scor_clean", "scor_quasi"]
    df = pd.read_csv(ruta_archivo, sep=r"\s+", header=None, names=columnas, dtype=int)
    return df["scor_clean"].tolist()

print("Funciones de carga definidas.")

Funciones de carga definidas.


In [4]:
def detectar_ciclos_rem(secuencia, fase_rem=4, umbral_separacion_rem=20, umbral_ciclo=60, epocas_por_minuto=2):
    """
    Detecta ciclos de sueño de forma robusta agrupando REM.
    """
    # Busca la fase indicada (por defecto 4, que será REM tras remapear)
    indices_rem = [i for i, x in enumerate(secuencia) if x == fase_rem]
    
    if not indices_rem:
        return []

    # Consolidar bloques REM
    periodos_rem = []
    if indices_rem:
        inicio_actual = indices_rem[0]
        fin_actual = indices_rem[0]
        
        for i in range(1, len(indices_rem)):
            distancia = indices_rem[i] - indices_rem[i-1]
            if distancia <= (umbral_separacion_rem * epocas_por_minuto):
                fin_actual = indices_rem[i]
            else:
                periodos_rem.append((inicio_actual, fin_actual))
                inicio_actual = indices_rem[i]
                fin_actual = indices_rem[i]
        periodos_rem.append((inicio_actual, fin_actual))

    # Definir ciclos
    ciclos = []
    inicio_ciclo = 0
    for p_inicio, p_fin in periodos_rem:
        ciclos.append((inicio_ciclo, p_fin))
        inicio_ciclo = p_fin + 1
        
    if inicio_ciclo < len(secuencia):
        ciclos.append((inicio_ciclo, len(secuencia) - 1))

    return ciclos

print("Función de detección de ciclos definida.")

Función de detección de ciclos definida.


In [5]:
# Cargar todos los hipnogramas y aplicar agrupación
pacientes = listar_pacientes()
print(f"Pacientes detectados ({len(pacientes)}): {', '.join(pacientes)}")

hipnogramas_agrupados = {}
for paciente in pacientes:
    raw_seq = cargar_hipnograma(paciente)
    seq_agrupada = remapear_secuencia(raw_seq)
    hipnogramas_agrupados[paciente] = seq_agrupada

print(f"\nTotal de épocas procesadas: {sum(len(seq) for seq in hipnogramas_agrupados.values())}")

Pacientes detectados (10): AR_2, DG_2, EL_2, GA_2, IN_2, JS_2, LL_2, SS_2, VB_2, VC_2

Total de épocas procesadas: 9501


In [6]:
def extraer_bigramas(secuencia):
    """Extrae todos los bigramas (pares consecutivos) de una secuencia."""
    bigramas = []
    for i in range(len(secuencia) - 1):
        bigramas.append((secuencia[i], secuencia[i+1]))
    return bigramas


def construir_matriz_transicion(bigramas):
    """
    Construye la matriz de transición (probabilidades condicionales) a partir de bigramas.
    """
    if not bigramas:
        return None
    
    # Obtener todas las fases únicas presentes en los bigramas
    fases_unicas = sorted(set([fase for bigrama in bigramas for fase in bigrama]))
    
    if not fases_unicas:
        return None
    
    # Inicializar matriz de conteos absolutos
    matriz_absoluta = pd.DataFrame(
        0, 
        index=fases_unicas, 
        columns=fases_unicas, 
        dtype=int
    )
    
    # Contar bigramas
    contador = Counter(bigramas)
    for (desde, hacia), count in contador.items():
        if desde in matriz_absoluta.index and hacia in matriz_absoluta.columns:
            matriz_absoluta.loc[desde, hacia] = count
    
    # Calcular probabilidades condicionales (normalizar por fila)
    suma_por_fila = matriz_absoluta.sum(axis=1)
    matriz_probabilidades = matriz_absoluta.div(suma_por_fila, axis=0).fillna(0)
    
    return matriz_probabilidades


def construir_matriz_sin_diagonal(matriz_probabilidades):
    """
    Construye una versión de la matriz donde la diagonal (quedarse en la misma fase) es cero,
    y luego re-normaliza por fila.
    
    Objetivo: Ver claramente hacia dónde se va el cerebro cuando decide abandonar una fase,
    sin que el gráfico se 'coma' el color por la alta probabilidad de quedarse en la misma fase.
    """
    # Copiar la matriz
    matriz_sin_diag = matriz_probabilidades.copy()
    
    # Poner la diagonal en cero
    for fase in matriz_sin_diag.index:
        if fase in matriz_sin_diag.columns:
            matriz_sin_diag.loc[fase, fase] = 0
    
    # Re-normalizar por fila (suma de cada fila debe ser 1)
    suma_por_fila = matriz_sin_diag.sum(axis=1)
    # Evitar división por cero (si una fila tiene suma 0, mantenerla en 0)
    matriz_sin_diag = matriz_sin_diag.div(suma_por_fila, axis=0).fillna(0)
    
    return matriz_sin_diag

print("Funciones de extracción y construcción de matrices definidas.")

Funciones de extracción y construcción de matrices definidas.


In [7]:
# Extraer bigramas por ciclo para todos los pacientes
bigramas_por_ciclo = defaultdict(list)  # {numero_ciclo: [lista de bigramas de todos los pacientes]}

for paciente, secuencia in hipnogramas_agrupados.items():
    # Detectar ciclos en la secuencia agrupada (REM=4)
    ciclos = detectar_ciclos_rem(secuencia, fase_rem=4)
    
    if not ciclos:
        continue
    
    # Extraer bigramas de cada ciclo
    for num_ciclo, (inicio, fin) in enumerate(ciclos, start=1):
        ciclo_seq = secuencia[inicio:fin+1]
        bigramas_ciclo = extraer_bigramas(ciclo_seq)
        bigramas_por_ciclo[num_ciclo].extend(bigramas_ciclo)

print(f"Ciclos detectados: {sorted(bigramas_por_ciclo.keys())}")
print(f"Total de bigramas por ciclo:")
for ciclo_num in sorted(bigramas_por_ciclo.keys()):
    print(f"  Ciclo {ciclo_num}: {len(bigramas_por_ciclo[ciclo_num])} bigramas")

Ciclos detectados: [1, 2, 3, 4, 5, 6, 7]
Total de bigramas por ciclo:
  Ciclo 1: 2215 bigramas
  Ciclo 2: 2062 bigramas
  Ciclo 3: 1914 bigramas
  Ciclo 4: 1656 bigramas
  Ciclo 5: 1292 bigramas
  Ciclo 6: 273 bigramas
  Ciclo 7: 33 bigramas


In [8]:
def graficar_heatmap_ciclo(matriz_probabilidades, titulo, ciclo_num, mostrar_diagonal=True):
    """
    Genera un heatmap interactivo con Plotly para la matriz de transición de un ciclo.
    
    Args:
        matriz_probabilidades: DataFrame con probabilidades condicionales
        titulo: Título del gráfico
        ciclo_num: Número del ciclo
        mostrar_diagonal: Si False, oculta visualmente la diagonal (poner texto vacío)
    """
    # Preparar etiquetas
    etiquetas_filas = [ETIQUETAS_FASES.get(fase, f"Fase {fase}") for fase in matriz_probabilidades.index]
    etiquetas_columnas = [ETIQUETAS_FASES.get(fase, f"Fase {fase}") for fase in matriz_probabilidades.columns]
    
    # Preparar valores y textos
    valores = matriz_probabilidades.values
    # Mostrar todos los valores, incluyendo los que son exactamente 0
    textos = [[f"{valor:.3f}" for valor in fila] for fila in valores]
    
    # Si no queremos mostrar la diagonal, ocultar el texto pero mantener el valor 0
    if not mostrar_diagonal:
        valores_plot = valores.copy()
        for i, fase in enumerate(matriz_probabilidades.index):
            if fase in matriz_probabilidades.columns:
                j = matriz_probabilidades.columns.get_loc(fase)
                # No poner texto en la diagonal
                textos[i][j] = ""
    else:
        valores_plot = valores
    
    # Crear heatmap
    fig = go.Figure(
        data=go.Heatmap(
            z=valores_plot,
            x=etiquetas_columnas,
            y=etiquetas_filas,
            text=textos,
            texttemplate="%{text}",
            textfont=dict(size=11, color="white"),
            colorscale="Cividis",
            zmin=0,
            zmax=1,
            colorbar=dict(
                title=dict(text="Probabilidad", font=dict(color="white")),
                tickfont=dict(color="white")
            ),
            hovertemplate="<b>Desde:</b> %{y}<br><b>Hacia:</b> %{x}<br><b>Probabilidad:</b> %{z:.4f}<extra></extra>"
        )
    )
    
    fig.update_layout(
        template="plotly_dark",
        paper_bgcolor="black",
        plot_bgcolor="black",
        title=dict(
            text=f"<b>{titulo}</b><br><sub>P(Fase siguiente | Fase actual)</sub>",
            font=dict(size=16, color="white"),
            x=0.5
        ),
        xaxis=dict(
            title=dict(text="Fase siguiente", font=dict(color="white")),
            tickfont=dict(color="white")
        ),
        yaxis=dict(
            title=dict(text="Fase actual", font=dict(color="white")),
            tickfont=dict(color="white"),
            autorange="reversed"  # Invertir eje Y
        ),
        width=900,
        height=700
    )
    
    return fig

print("Función de visualización definida.")

Función de visualización definida.


In [9]:
# Construir matrices de transición para cada ciclo
matrices_por_ciclo = {}

for ciclo_num in sorted(bigramas_por_ciclo.keys()):
    bigramas = bigramas_por_ciclo[ciclo_num]
    matriz = construir_matriz_transicion(bigramas)
    if matriz is not None:
        matrices_por_ciclo[ciclo_num] = matriz
        print(f"\nCiclo {ciclo_num}: Matriz {matriz.shape[0]}x{matriz.shape[1]}")
    else:
        print(f"\nCiclo {ciclo_num}: Sin datos suficientes")

print(f"\nTotal de matrices construidas: {len(matrices_por_ciclo)}")


Ciclo 1: Matriz 6x6

Ciclo 2: Matriz 6x6

Ciclo 3: Matriz 6x6

Ciclo 4: Matriz 6x6

Ciclo 5: Matriz 6x6

Ciclo 6: Matriz 6x6

Ciclo 7: Matriz 4x4

Total de matrices construidas: 7


In [10]:
# Construir matrices sin diagonal para cada ciclo
matrices_sin_diagonal_por_ciclo = {}

for ciclo_num in sorted(matrices_por_ciclo.keys()):
    matriz = matrices_por_ciclo[ciclo_num]
    matriz_sin_diag = construir_matriz_sin_diagonal(matriz)
    matrices_sin_diagonal_por_ciclo[ciclo_num] = matriz_sin_diag
    print(f"Ciclo {ciclo_num}: Matriz sin diagonal construida")

print(f"\nTotal de matrices sin diagonal: {len(matrices_sin_diagonal_por_ciclo)}")

Ciclo 1: Matriz sin diagonal construida
Ciclo 2: Matriz sin diagonal construida
Ciclo 3: Matriz sin diagonal construida
Ciclo 4: Matriz sin diagonal construida
Ciclo 5: Matriz sin diagonal construida
Ciclo 6: Matriz sin diagonal construida
Ciclo 7: Matriz sin diagonal construida

Total de matrices sin diagonal: 7


## Análisis: Evolución de Transiciones Clave

Analizamos cómo cambian las probabilidades de transición desde S2 hacia REM y S3/S4 a lo largo de los ciclos.

## Visualización: Matrices CON Diagonal por Ciclo

Visualizamos las matrices de transición estándar (con diagonal) para los primeros ciclos. La diagonal muestra la probabilidad de quedarse en la misma fase, que suele ser alta (0.84-0.91 para S2).


In [16]:
# Graficar los primeros 7 ciclos CON diagonal (que son los que existen)
ciclos_a_mostrar = sorted(matrices_por_ciclo.keys())[:7]

for ciclo_num in ciclos_a_mostrar:
    matriz = matrices_por_ciclo[ciclo_num]
    fig = graficar_heatmap_ciclo(
        matriz,
        f"Matriz de Transición - Ciclo {ciclo_num} (Con Diagonal)",
        ciclo_num,
        mostrar_diagonal=True
    )
    fig.show()


## Visualización: Matrices Sin Diagonal por Ciclo

Esta versión elimina la probabilidad de quedarse en la misma fase (diagonal = 0) y re-normaliza por fila. Esto permite ver claramente **hacia dónde se va el cerebro cuando decide abandonar una fase**, sin que el gráfico se "coma" el color por la alta probabilidad de quedarse en la misma fase.

**Objetivo**: Ver cómo resalta la transición S2 → S3/S4 en el Ciclo 1 (amarillo brillante) y cómo ese amarillo se apaga en los ciclos siguientes.

In [12]:
# Graficar los ciclos
ciclos_a_mostrar = sorted(matrices_sin_diagonal_por_ciclo.keys())

for ciclo_num in ciclos_a_mostrar:
    matriz_sin_diag = matrices_sin_diagonal_por_ciclo[ciclo_num]
    fig = graficar_heatmap_ciclo(
        matriz_sin_diag,
        f"Matriz de Transición - Ciclo {ciclo_num} (Sin Diagonal)",
        ciclo_num,
        mostrar_diagonal=False
    )
    fig.show()

In [13]:
# Extraer probabilidades de transición S2 → REM y S2 → S3/S4 por ciclo
# Tanto de las matrices CON diagonal como SIN diagonal
fase_s2 = 2
fase_rem = 4
fase_sws = 3

evolucion_transiciones = []
evolucion_transiciones_sin_diag = []

for ciclo_num in sorted(matrices_por_ciclo.keys()):
    matriz = matrices_por_ciclo[ciclo_num]
    matriz_sin_diag = matrices_sin_diagonal_por_ciclo.get(ciclo_num)
    
    # Extraer de matriz CON diagonal
    if fase_s2 in matriz.index:
        prob_s2_rem = matriz.loc[fase_s2, fase_rem] if fase_rem in matriz.columns else 0.0
        prob_s2_sws = matriz.loc[fase_s2, fase_sws] if fase_sws in matriz.columns else 0.0
        prob_s2_quedarse = matriz.loc[fase_s2, fase_s2] if fase_s2 in matriz.columns else 0.0
        
        evolucion_transiciones.append({
            "Ciclo": ciclo_num,
            "S2 → REM": prob_s2_rem,
            "S2 → S3/S4": prob_s2_sws,
            "S2 → S2 (quedarse)": prob_s2_quedarse
        })
    
    # Extraer de matriz SIN diagonal
    if matriz_sin_diag is not None and fase_s2 in matriz_sin_diag.index:
        prob_s2_rem_sd = matriz_sin_diag.loc[fase_s2, fase_rem] if fase_rem in matriz_sin_diag.columns else 0.0
        prob_s2_sws_sd = matriz_sin_diag.loc[fase_s2, fase_sws] if fase_sws in matriz_sin_diag.columns else 0.0
        
        evolucion_transiciones_sin_diag.append({
            "Ciclo": ciclo_num,
            "S2 → REM (sin diag)": prob_s2_rem_sd,
            "S2 → S3/S4 (sin diag)": prob_s2_sws_sd
        })

df_evolucion = pd.DataFrame(evolucion_transiciones)
df_evolucion_sin_diag = pd.DataFrame(evolucion_transiciones_sin_diag)

# Combinar ambos dataframes
if not df_evolucion_sin_diag.empty:
    df_evolucion_completo = pd.merge(df_evolucion, df_evolucion_sin_diag, on="Ciclo", how="outer")
else:
    df_evolucion_completo = df_evolucion

print("Evolución de transiciones desde S2 (CON diagonal):")
print(df_evolucion.to_string(index=False))
print("\n" + "="*60)
print("Evolución de transiciones desde S2 (SIN diagonal - renormalizado):")
print(df_evolucion_sin_diag.to_string(index=False))
print("\n" + "="*60)
print("Resumen comparativo:")
print(df_evolucion_completo.to_string(index=False))

Evolución de transiciones desde S2 (CON diagonal):
 Ciclo  S2 → REM  S2 → S3/S4  S2 → S2 (quedarse)
     1  0.022400    0.107200            0.841600
     2  0.025612    0.060134            0.870824
     3  0.014274    0.044500            0.897565
     4  0.012346    0.034792            0.911336
     5  0.005487    0.038409            0.906722
     6  0.040323    0.024194            0.846774
     7  0.000000    0.000000            0.846154

Evolución de transiciones desde S2 (SIN diagonal - renormalizado):
 Ciclo  S2 → REM (sin diag)  S2 → S3/S4 (sin diag)
     1             0.141414               0.676768
     2             0.198276               0.465517
     3             0.139344               0.434426
     4             0.139241               0.392405
     5             0.058824               0.411765
     6             0.263158               0.157895
     7             0.000000               0.000000

Resumen comparativo:
 Ciclo  S2 → REM  S2 → S3/S4  S2 → S2 (quedarse)  S2 → REM 

In [14]:
# Visualizar la evolución de las transiciones (CON y SIN diagonal)
fig = go.Figure()

# Líneas CON diagonal (sólidas)
# S2 → REM (con diagonal)
fig.add_trace(go.Scatter(
    x=df_evolucion["Ciclo"],
    y=df_evolucion["S2 → REM"],
    mode='lines+markers',
    name='S2 → REM (con diag)',
    line=dict(color='#AB63FA', width=3),
    marker=dict(size=10, color='#AB63FA'),
    hovertemplate="<b>Ciclo:</b> %{x}<br><b>Probabilidad (con diag):</b> %{y:.4f}<extra></extra>"
))

# S2 → S3/S4 (con diagonal)
fig.add_trace(go.Scatter(
    x=df_evolucion["Ciclo"],
    y=df_evolucion["S2 → S3/S4"],
    mode='lines+markers',
    name='S2 → S3/S4 (con diag)',
    line=dict(color='#00CC96', width=3),
    marker=dict(size=10, color='#00CC96'),
    hovertemplate="<b>Ciclo:</b> %{x}<br><b>Probabilidad (con diag):</b> %{y:.4f}<extra></extra>"
))

# Líneas SIN diagonal (punteadas, más gruesas para destacar)
if not df_evolucion_sin_diag.empty:
    # S2 → REM (sin diagonal)
    fig.add_trace(go.Scatter(
        x=df_evolucion_sin_diag["Ciclo"],
        y=df_evolucion_sin_diag["S2 → REM (sin diag)"],
        mode='lines+markers',
        name='S2 → REM (sin diag)',
        line=dict(color='#FF6B9D', width=3, dash='dash'),
        marker=dict(size=10, color='#FF6B9D', symbol='diamond'),
        hovertemplate="<b>Ciclo:</b> %{x}<br><b>Probabilidad (sin diag):</b> %{y:.4f}<extra></extra>"
    ))
    
    # S2 → S3/S4 (sin diagonal)
    fig.add_trace(go.Scatter(
        x=df_evolucion_sin_diag["Ciclo"],
        y=df_evolucion_sin_diag["S2 → S3/S4 (sin diag)"],
        mode='lines+markers',
        name='S2 → S3/S4 (sin diag)',
        line=dict(color='#FFD93D', width=3, dash='dash'),
        marker=dict(size=10, color='#FFD93D', symbol='diamond'),
        hovertemplate="<b>Ciclo:</b> %{x}<br><b>Probabilidad (sin diag):</b> %{y:.4f}<extra></extra>"
    ))

fig.update_layout(
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    title=dict(
        text="<b>Evolución de Transiciones desde S2</b><br><sub>Comparación: CON diagonal (sólido) vs SIN diagonal (punteado, renormalizado)</sub>",
        font=dict(size=16, color="white"),
        x=0.5
    ),
    xaxis=dict(
        title=dict(text="Número de Ciclo", font=dict(color="white")),
        tickfont=dict(color="white")
    ),
    yaxis=dict(
        title=dict(text="Probabilidad de Transición", font=dict(color="white")),
        tickfont=dict(color="white")
    ),
    width=1000,
    height=600,
    legend=dict(
        title=dict(text="Transición", font=dict(color="white")),
        font=dict(color="white"),
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

## Visualización Comparativa: Matrices por Ciclo

Visualizamos las matrices de transición para los primeros ciclos para comparar los patrones.

In [15]:
# Extraer más transiciones clave por ciclo (CON y SIN diagonal)
resumen_ciclos = []
resumen_ciclos_sin_diag = []

for ciclo_num in sorted(matrices_por_ciclo.keys()):
    matriz = matrices_por_ciclo[ciclo_num]
    matriz_sin_diag = matrices_sin_diagonal_por_ciclo.get(ciclo_num)
    
    # Resumen CON diagonal
    resumen = {"Ciclo": ciclo_num}
    
    # Transiciones desde S2
    if fase_s2 in matriz.index:
        resumen["S2 → REM"] = matriz.loc[fase_s2, fase_rem] if fase_rem in matriz.columns else 0.0
        resumen["S2 → S3/S4"] = matriz.loc[fase_s2, fase_sws] if fase_sws in matriz.columns else 0.0
        resumen["S2 → S2"] = matriz.loc[fase_s2, fase_s2] if fase_s2 in matriz.columns else 0.0
    
    # Transiciones desde S3/S4
    if fase_sws in matriz.index:
        resumen["S3/S4 → S2"] = matriz.loc[fase_sws, fase_s2] if fase_s2 in matriz.columns else 0.0
        resumen["S3/S4 → S3/S4"] = matriz.loc[fase_sws, fase_sws] if fase_sws in matriz.columns else 0.0
    
    # Transiciones desde REM
    if fase_rem in matriz.index:
        resumen["REM → S2"] = matriz.loc[fase_rem, fase_s2] if fase_s2 in matriz.columns else 0.0
        resumen["REM → REM"] = matriz.loc[fase_rem, fase_rem] if fase_rem in matriz.columns else 0.0
    
    resumen_ciclos.append(resumen)
    
    # Resumen SIN diagonal
    if matriz_sin_diag is not None:
        resumen_sd = {"Ciclo": ciclo_num}
        
        # Transiciones desde S2
        if fase_s2 in matriz_sin_diag.index:
            resumen_sd["S2 → REM (SD)"] = matriz_sin_diag.loc[fase_s2, fase_rem] if fase_rem in matriz_sin_diag.columns else 0.0
            resumen_sd["S2 → S3/S4 (SD)"] = matriz_sin_diag.loc[fase_s2, fase_sws] if fase_sws in matriz_sin_diag.columns else 0.0
        
        # Transiciones desde S3/S4
        if fase_sws in matriz_sin_diag.index:
            resumen_sd["S3/S4 → S2 (SD)"] = matriz_sin_diag.loc[fase_sws, fase_s2] if fase_s2 in matriz_sin_diag.columns else 0.0
            resumen_sd["S3/S4 → S3/S4 (SD)"] = matriz_sin_diag.loc[fase_sws, fase_sws] if fase_sws in matriz_sin_diag.columns else 0.0
        
        # Transiciones desde REM
        if fase_rem in matriz_sin_diag.index:
            resumen_sd["REM → S2 (SD)"] = matriz_sin_diag.loc[fase_rem, fase_s2] if fase_s2 in matriz_sin_diag.columns else 0.0
            resumen_sd["REM → REM (SD)"] = matriz_sin_diag.loc[fase_rem, fase_rem] if fase_rem in matriz_sin_diag.columns else 0.0
        
        resumen_ciclos_sin_diag.append(resumen_sd)

df_resumen = pd.DataFrame(resumen_ciclos)
df_resumen_sd = pd.DataFrame(resumen_ciclos_sin_diag)

# Combinar ambos resúmenes
if not df_resumen_sd.empty:
    df_resumen_completo = pd.merge(df_resumen, df_resumen_sd, on="Ciclo", how="outer")
else:
    df_resumen_completo = df_resumen

print("Resumen de probabilidades por ciclo (CON diagonal):\n")
print(df_resumen.to_string(index=False))
print("\nResumen de probabilidades por ciclo (SIN diagonal - renormalizado):\n")
print(df_resumen_sd.to_string(index=False))
print("\nResumen comparativo completo:\n")
print(df_resumen_completo.to_string(index=False))

# Análisis de tendencias (usando ambas versiones)
if len(df_resumen) > 1:
    print("\nAnálisis de Tendencias (CON diagonal)\n")
    
    # Calcular correlación entre número de ciclo y probabilidad
    if "S2 → REM" in df_resumen.columns:
        correlacion_rem = np.corrcoef(df_resumen["Ciclo"], df_resumen["S2 → REM"])[0, 1]
        print(f"Correlación Ciclo vs S2→REM: {correlacion_rem:.3f}")
        if correlacion_rem > 0:
            print("  → Tendencia: Aumenta la probabilidad de ir a REM en ciclos posteriores")
        elif correlacion_rem < 0:
            print("  → Tendencia: Disminuye la probabilidad de ir a REM en ciclos posteriores")
        else:
            print("  → Tendencia: Sin cambio significativo")
    
    if "S2 → S3/S4" in df_resumen.columns:
        correlacion_sws = np.corrcoef(df_resumen["Ciclo"], df_resumen["S2 → S3/S4"])[0, 1]
        print(f"Correlación Ciclo vs S2→S3/S4: {correlacion_sws:.3f}")
        if correlacion_sws > 0:
            print("  → Tendencia: Aumenta la probabilidad de ir a S3/S4 en ciclos posteriores")
        elif correlacion_sws < 0:
            print("  → Tendencia: Disminuye la probabilidad de ir a S3/S4 en ciclos posteriores")
        else:
            print("  → Tendencia: Sin cambio significativo")
    
    # Análisis de tendencias SIN diagonal
    if not df_resumen_sd.empty and len(df_resumen_sd) > 1:
        print("\nAnálisis de Tendencias (SIN diagonal - renormalizado)\n")
        
        if "S2 → REM (SD)" in df_resumen_sd.columns:
            correlacion_rem_sd = np.corrcoef(df_resumen_sd["Ciclo"], df_resumen_sd["S2 → REM (SD)"])[0, 1]
            print(f"Correlación Ciclo vs S2→REM (sin diag): {correlacion_rem_sd:.3f}")
            if correlacion_rem_sd > 0:
                print("  → Tendencia: Aumenta la probabilidad de ir a REM en ciclos posteriores")
            elif correlacion_rem_sd < 0:
                print("  → Tendencia: Disminuye la probabilidad de ir a REM en ciclos posteriores")
            else:
                print("  → Tendencia: Sin cambio significativo")
        
        if "S2 → S3/S4 (SD)" in df_resumen_sd.columns:
            correlacion_sws_sd = np.corrcoef(df_resumen_sd["Ciclo"], df_resumen_sd["S2 → S3/S4 (SD)"])[0, 1]
            print(f"Correlación Ciclo vs S2→S3/S4 (sin diag): {correlacion_sws_sd:.3f}")
            if correlacion_sws_sd > 0:
                print("  → Tendencia: Aumenta la probabilidad de ir a S3/S4 en ciclos posteriores")
            elif correlacion_sws_sd < 0:
                print("  → Tendencia: Disminuye la probabilidad de ir a S3/S4 en ciclos posteriores")
            else:
                print("  → Tendencia: Sin cambio significativo")

Resumen de probabilidades por ciclo (CON diagonal):

 Ciclo  S2 → REM  S2 → S3/S4  S2 → S2  S3/S4 → S2  S3/S4 → S3/S4  REM → S2  REM → REM
     1  0.022400    0.107200 0.841600    0.055498       0.930113  0.008333   0.979167
     2  0.025612    0.060134 0.870824    0.064286       0.901786  0.024590   0.959016
     3  0.014274    0.044500 0.897565    0.252809       0.702247  0.011933   0.964200
     4  0.012346    0.034792 0.911336    0.208000       0.736000  0.004000   0.972000
     5  0.005487    0.038409 0.906722    0.337662       0.636364  0.000000   0.993750
     6  0.040323    0.024194 0.846774    0.040000       0.960000  0.062500   0.906250
     7  0.000000    0.000000 0.846154         NaN            NaN       NaN        NaN

Resumen de probabilidades por ciclo (SIN diagonal - renormalizado):

 Ciclo  S2 → REM (SD)  S2 → S3/S4 (SD)  S3/S4 → S2 (SD)  S3/S4 → S3/S4 (SD)  REM → S2 (SD)  REM → REM (SD)
     1       0.141414         0.676768         0.794118                 0.0       